In [1]:
from sagas.nlu.ruleset_procs import list_words, cached_chunks, get_main_domains
from sagas.conf.conf import cf
from sagas.tracker_jupyter import enable_jupyter_tracker
enable_jupyter_tracker()

# She denied being my mother
sents='Ela negou ser minha mãe.'
lang='pt'
domain='verb_domains'
chunks = cached_chunks(sents, lang, cf.engine(lang))
domains = chunks[domain]
domains

[{'index': 2,
  'text': 'negou',
  'lemma': 'negar',
  'upos': 'VERB',
  'xpos': '_',
  'feats': 'Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin',
  'governor': 0,
  'dependency_relation': 'root',
  'entity': [],
  'segments': [],
  'obl': [{'index': 5,
    'text': 'mãe',
    'lemma': 'mãe',
    'upos': 'NOUN',
    'xpos': '_',
    'feats': 'Gender=Fem|Number=Sing',
    'governor': 2,
    'dependency_relation': 'obl',
    'entity': [],
    'segments': [],
    'det': [{'index': 4,
      'text': 'minha',
      'lemma': 'meu',
      'upos': 'DET',
      'xpos': '_',
      'feats': 'Gender=Fem|Number=Sing|PronType=Prs',
      'governor': 5,
      'dependency_relation': 'det',
      'entity': [],
      'segments': []}],
    'cop': [{'index': 3,
      'text': 'ser',
      'lemma': 'ser',
      'upos': 'AUX',
      'xpos': '_',
      'feats': 'VerbForm=Inf',
      'governor': 5,
      'dependency_relation': 'cop',
      'entity': [],
      'segments': []}]}],
  'punct': [{'index': 6,
 

In [2]:
[w.text for w in chunks['doc'].words]

['Ela', 'negou', 'ser', 'minha', 'mãe', '.']

In [2]:
from sagas.nlu.uni_chunks import get_chunk
import sagas

cla='obl'
result=get_chunk(chunks, domain, cla, lambda w: sagas.to_obj({'rel':w.dependency_relation, 
                                                 'pos':w.upos.lower(), 
                                                 'word':f"{w.text}/{w.lemma}"}))
result

[{'rel': 'cop', 'pos': 'aux', 'word': 'ser/ser'},
 {'rel': 'det', 'pos': 'det', 'word': 'minha/meu'},
 {'rel': 'obl', 'pos': 'noun', 'word': 'mãe/mãe'}]

In [37]:
from dataclasses import dataclass
@dataclass
class WordData:
    index: int
    rel: str
    pos: str
    word: str    
        
cla='obl'
ana=get_chunk(chunks, domain, cla, lambda w: WordData(index=w.index,
                                               rel=w.dependency_relation, 
                                               pos=w.upos.lower(), 
                                               word=f"{w.text}/{w.lemma}"))
# {e['index']:e for e in ana}
ana

[WordData(index=3, rel='cop', pos='aux', word='ser/ser'),
 WordData(index=4, rel='det', pos='det', word='minha/meu'),
 WordData(index=5, rel='obl', pos='noun', word='mãe/mãe')]

In [38]:
ana[0]['index']

TypeError: 'WordData' object is not subscriptable

In [12]:
from pampy import match, _
# word_data={e['index']:e for e in ana}
for word_data in ana:
    r=match(word_data, 
          WordData(_,_, 'aux', _), lambda *arg: f"aux: {arg[2]}",
          WordData(_,'obl','noun', _), lambda *arg: arg,
          _,              None
         ) 
    print(r)

aux: ser/ser
None
(5, 'mãe/mãe')


In [20]:
cla='obl'
raw=get_chunk(chunks, domain, cla, lambda w: {'rel':w.dependency_relation, 
                                              'pos':w.upos.lower(), 
                                              'word':f"{w.text}/{w.lemma}"})
{e['rel']:e for e in raw}

{'cop': {'rel': 'cop', 'pos': 'aux', 'word': 'ser/ser'},
 'det': {'rel': 'det', 'pos': 'det', 'word': 'minha/meu'},
 'obl': {'rel': 'obl', 'pos': 'noun', 'word': 'mãe/mãe'}}

In [27]:
rs={e['rel']:e for e in raw}
match(rs, { 'cop': { 'word': _ }, 'obl': {'pos':'noun', 'word':_} }, lambda *arg: arg,
      _,              "anything else"
     )        # => 3

('ser/ser', 'mãe/mãe')

In [36]:
rs={e['rel']:e for e in raw}
match(rs, { _: { 'pos': 'aux' }, 'obl': {'pos':'noun', 'word':_} }, lambda *arg: arg,
      _,              "anything else"
     )        # => 3

('cop', 'mãe/mãe')

In [35]:
rs={e['pos']:e for e in raw}
print(rs)
match(rs, { 'aux': { 'rel': _ }, 'noun': {'rel': 'obl', 'word':_} }, lambda *arg: arg,
      _,              "anything else"
     )        # => 3

{'aux': {'rel': 'cop', 'pos': 'aux', 'word': 'ser/ser'}, 'det': {'rel': 'det', 'pos': 'det', 'word': 'minha/meu'}, 'noun': {'rel': 'obl', 'pos': 'noun', 'word': 'mãe/mãe'}}


('cop', 'mãe/mãe')

In [4]:
from sagas.nlu.nlu_cli import retrieve_word_info
rel='cop'
word=next((w.word for w in result if w.rel==rel), None)
if word:
    print(word)
    rs = retrieve_word_info('get_synsets', word, lang, '*')
    print(rs)

ser/ser
['entity.n.01', 'organism.n.01', 'animal.n.01', 'debut.v.03', 'exist.v.01', 'be.v.01', 'creature.n.02', 'being.n.01']


In [5]:
def check_chain(kind, word, pos, lang):
    from sagas.nlu.synonyms import synonyms
    from sagas.nlu.utils import predicate
    
    r=synonyms.match(word, lang)
    if r is None:
        return predicate(kind, word, lang, pos)
    return predicate(kind, r, 'en', pos)

check_chain('be', word, '*', lang)

True

In [6]:
cats={'be'}
any([check_chain(cat, word, '*', lang) for cat in cats])

True

In [7]:
from sagas.nlu.inspector_clauses import check_clause_sub
check_clause_sub(sents, 'pt', 'verb_domains', 'obl', 'cop', {'be'})

True

In [3]:
from sagas.nlu.inspector_common import cla_meta
from sagas.nlu.inspector_clauses import ClausesInspector as clauses, cla_expr

e=cla_expr('verb:obl', cop={'be'})
e.run('', cla_meta('Ela negou ser minha mãe.', 'pt'))

True

In [6]:
sents='建物の裏に駐車しました。'
lang='ja'
# domain='doc'
domain='predicts'
chunks = cached_chunks(sents, lang, cf.engine(lang))
domains = chunks[domain]
domains

[{'type': 'predicate',
  'lemma': '駐車',
  'index': 2,
  'phonetic': 'ちゅうしゃ',
  'word': '駐車しました。',
  'rel': 'D',
  'governor': 0,
  'pos': 'noun',
  'domains': [['ニ', 1, '裏に', '裏', ['裏'], ['c_noun', 'x_n']]],
  'stems': [],
  'segments': [{'index': 4,
    'upos': 'NOUN',
    'xpos': ['名詞', 'サ変名詞'],
    'text': '駐車',
    'lemmas': ['駐車', 'ちゅうしゃ']},
   {'index': 5,
    'upos': 'VERB',
    'xpos': ['動詞', '*'],
    'text': 'し',
    'lemmas': ['する', 'する']},
   {'index': 6,
    'upos': 'X',
    'xpos': ['接尾辞', '動詞性接尾辞'],
    'text': 'ました',
    'lemmas': ['ます', 'ます']},
   {'index': 7, 'upos': 'PUNCT', 'xpos': ['特殊', '句点'], 'text': '。'}]}]

In [7]:
chunks['doc'].as_json

[{'index': 1,
  'text': '建物の',
  'lemma': '建物',
  'upos': 'NOUN',
  'xpos': 'n_p',
  'feats': ['<文頭><助詞><連体修飾><体言><係:ノ格><区切:0-4><名詞項候補><先行詞候補><係チ:非用言格解析||用言&&文節内:Ｔ解析格-ヲ><正規化代表表記:建物/たてもの>'],
  'governor': 2,
  'dependency_relation': 'D',
  'entity': [],
  'segments': [{'index': 0,
    'upos': 'NOUN',
    'xpos': ['名詞', '普通名詞'],
    'text': '建物',
    'lemmas': ['建物', 'たてもの']},
   {'index': 1, 'upos': 'SCONJ', 'xpos': ['助詞', '接続助詞'], 'text': 'の'}]},
 {'index': 2,
  'text': '裏に',
  'lemma': '裏',
  'upos': 'NOUN',
  'xpos': 'n_p',
  'feats': ['<ニ><助詞><体言><一文字漢字><係:ニ格><区切:0-0><格要素><連用要素><名詞項候補><先行詞候補><正規化代表表記:裏/り?裏/うら><解析格:ニ>'],
  'governor': 3,
  'dependency_relation': 'ニ',
  'entity': [],
  'segments': [{'index': 2,
    'upos': 'NOUN',
    'xpos': ['名詞', '普通名詞'],
    'text': '裏',
    'lemmas': ['裏', 'り']},
   {'index': 3, 'upos': 'ADP', 'xpos': ['助詞', '格助詞'], 'text': 'に'}]},
 {'index': 3,
  'text': '駐車しました。',
  'lemma': '駐車',
  'upos': 'NOUN',
  'xpos': 'n_v_s_*',
  'feats': ['<文末><サ変動詞><時

In [16]:
from sagas.nlu.ruleset_procs import children
from sagas.nlu.nlu_cli import retrieve_word_info
from sagas.nlu.utils import get_possible_mean
sent=chunks['doc']
index=2
word = next(filter(lambda w: w.index == index, sent.words))
print(word.index, word.text)
for c in children(word, sent):
    word=f"{c.text}/{c.lemma}"
    rs = retrieve_word_info('get_synsets', word, 'ja', '*')
    mean = get_possible_mean(rs)
    print(mean, word)

2 裏に
building 建物の/建物


In [23]:
from sagas.nlu.parse_client_helper import build_context
from sagas.nlu.rules_header import *

data={'lang': lang, "sents": sents}
ctx,pat=next(build_context(data, 'predicts', name='_test_'))
index=next(x[1] for x in ctx.domains if x[0]=='ニ')
index+1

2